In [29]:
import pandas as pd
melb_df=pd.read_csv('data/melb_data_fe.csv', sep=',')

# МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# построение простой одномерной сводной таблицы по одному признаку: 
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median() # показывает зависимость медианной цены и площади здания от числа комнат

# Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type).
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.
melb_df.groupby(['Rooms', 'Type'])['Price'].mean() # сигнатура по двум признакам - ([1-ый уровень, 2-ой уровень])[искомый параметр]

# Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), 
# а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод unstack(), который позволяет переопределить вложенный индекс в виде столбцов таблицы:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

# МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table. В качестве параметра values укажем столбец Price, 
# в качестве индексов сводной таблицы возьмём Rooms, а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). 
# Дополнительно заменим пропуски в таблице на значение 0. Финальный результат для наглядности вывода округлим с помощью метода round() до целых.
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()
# Проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. 
# Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), 
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. 
# В качестве значений сводной таблицы возьмём количество продаж
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)
# Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). 
# Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. 
# Построим такую сводную таблицу, где пропущенные значения заменим на 0:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

# МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ !!!!!!!!!!!!!!!!!!!!!!!!!!
# Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), 
# а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):
melb_df.pivot_table(
    values='Price', # то что смотрим
    index=['Method','Type'], # по этим признакам группируем
    columns='Regionname', # разбивка по регионам
    aggfunc='median', # основной признак бьем по медиане, не по среднему
    fill_value=0 # нули тоже идут в сводник
)

# ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ !!!!!!!!!!!!!!!!
# сводник в переменную
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
# чтобы вывести данные:
pivot.columns
# чтобы вывести средние значения площадей участков для типа здания unit, просто нужно последовательно обратившись по имени столбцов
display(pivot['mean']['unit'])
# регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади
mask=pivot['mean']['house'] < pivot ['median']['house'] 
filtered_pivot=pivot[mask]
display(filtered_pivot)
# Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:
print(list(filtered_pivot.index))


# !!!!!!!!!!!!!!!!!!!! Мультииндексовые таблицы создаем вручную!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], # первый список задаёт внешний уровень вложенности
           ['up','down','right','up','down','up','down','left']]) # второй список — внутренний уровень вложенности
display(mser)
# Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:
print(mser.index)

# Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

# Задание 1.
# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). 
# Для какой комбинации признаков площадь здания наибольшая? В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.
melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
) # ответ house,7

# Задание 2.
# Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type). 
# Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.
pivot=melb_df.pivot_table( # снова shit up - в переменную не занес сводную таблицу чтобы в дальнейшем получить доступ к данным
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
)
max_unit_price=pivot['unit'].max() # объявил переменную в которую закинул из сводной таблицы максимальное значение по столбцу unit
print(pivot[pivot['unit'] == max_unit_price].index[0]) # вывел значение элемента в своднике по столбцу 0(название компании) которому соответствует максимальное значение, которое объявил в перемнной



Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']


white  up       0.735016
       down     0.360375
       right    0.873027
blue   up       0.246356
       down     0.579640
red    up       0.233418
       down     0.649557
       left     0.766564
dtype: float64

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )


pen               paper          
                   1         2         1         2
white up    1.159773 -1.227970  1.717277  0.970277
      down  0.928567  0.542659  2.421262 -0.672301
red   up    1.337430  0.561186 -0.534006 -0.447127
      down -1.190202 -0.493180 -0.356960 -1.217290

Nick
